In [ ]:
import pandas as pd

In [ ]:
import io
data = open('./tRNA-modifications.txt').read().split('#')
mod_df = pd.DataFrame()
for d in data:
    mod_df = pd.concat([mod_df, pd.read_csv(io.StringIO(d), delim_whitespace=True).T],join='outer')
all_paper_trnas = set([i.split('.')[0] for i in mod_df.index])

# Targets

In [ ]:
from Bio import SeqIO
trna_df = pd.read_csv('tRNAs.txt',sep='\t',index_col=0)
paper_dct = trna_df.reset_index().set_index('Codon')['tRNA'].to_dict()
gb_file = Bio.SeqIO.parse("../building_data/genome_modified.gb", "gb")
contigs = [ i for i in gb_file ]
dct = {'c':'G','g':'C','t':'A','a':'T'}

In [ ]:
gb_dct = {}
for c in contigs:
    for f in c.features:
        if f.type == 'tRNA':
            s = f.qualifiers['anticodon'][0].split('seq:')[1].split(')')[0][::-1]
            s2 = ''.join([dct[i] for i in s])
            i = f.qualifiers['old_locus_tag'][0]
            aa = f.qualifiers['product'][0].split('tRNA-')[1]
#             if 'Ala' in f.qualifiers['product'][0]:
            gb_dct[s2] = i
            print(i,aa,s2)

In [ ]:
paper_to_gb = {}
for c in gb_dct:
    gb_trna = gb_dct[c]
    paper_trna = paper_dct[c]
    paper_to_gb[paper_trna] = gb_trna

In [ ]:
coralme_rnamod = pd.DataFrame()

In [ ]:
for i in all_paper_trnas:
    if i not in paper_to_gb:
        print(i)
        continue
    trna = paper_to_gb[i]
    mods = mod_df[mod_df.index.str.contains(i)]
    rnamod_dct = {}
    for j,row in mods.iterrows():
        rnamod_dct[trna] = {'position':row[1],
                        'modification':row[0]}
        coralme_rnamod = pd.concat([coralme_rnamod, pd.DataFrame.from_dict(rnamod_dct).T],join='outer')

In [ ]:
coralme_rnamod

In [ ]:
# coralme_rnamod.to_csv('./building_data/post_transcriptional_modification_of_RNA.txt',sep='\t')

# Machinery

In [ ]:
dct = {}
for i,row in mod_df.iterrows():
    i = i.split('.')[0]
    if i not in paper_to_gb:
        print(i)
        continue
    if 'Monomer' not in row[2]:
        continue
    trna = paper_to_gb[i]
    enz = row[2].split('_assumed')[0]
    if enz not in dct:
        dct[enz] = set()
    dct[enz].add('{}_at_{}'.format(row[1],row[0]))


In [ ]:
for k,v in dct.items():
    mods = [i.split('_at_')[1] for i in v]
    pos = [i.split('_at_')[0] for i in v]
    print(k,mods,','.join(pos))

## ATPM

Supplementary information of the paper states atp maintenance is 25.0

https://static-content.springer.com/esm/art%3A10.1038%2Fncomms1928/MediaObjects/41467_2012_BFncomms1928_MOESM921_ESM.pdf

## New reactions

In [ ]:
import cobra

In [ ]:
model = cobra.io.read_sbml_model('./tmaritima-ME.xml')

In [ ]:
for r in model.metabolites.atp_c.reactions:
    if r.lower_bound > 0:
        print(r.id) 
        print(r.reaction)